In [1]:
from keras.models import Sequential

Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


XOR with Keras
=======

In [2]:
from keras.layers.core import Dense, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD

model = Sequential()

model.add(Dense(2, input_dim=2))
model.add(Activation('sigmoid'))
model.add(Dense(1))
model.add(Activation('sigmoid'))


sgd = SGD(lr=0.1, decay=1e-6, momentum=0.99, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)

Generate some XOR input data and test data

In [30]:
import numpy as np

nsamples = 100000

xs = np.empty([nsamples, 2], dtype='uint8')
ys = np.empty([nsamples], dtype='uint8')
for i in range(nsamples):
    xs[i, 0] = np.random.randint(0, 2)
    xs[i, 1] = np.random.randint(0, 2)
    ys[i] = np.logical_xor(xs[i, 0], xs[i, 1])
    
    
nsamples = 1000
testxs = np.empty([nsamples, 2], dtype='uint8')
testys = np.empty([nsamples, 1], dtype='uint8')
for i in range(nsamples):
    testxs[i, 0] = np.random.randint(0, 2)
    testxs[i, 1] = np.random.randint(0, 2)
    testys[i, 0] = np.logical_xor(testxs[i, 0], testxs[i, 1])

train

In [31]:
model.fit(xs, ys,
          nb_epoch=5,
          batch_size=32,
          show_accuracy=True,
          validation_data=(testxs, testys))

Train on 100000 samples, validate on 1000 samples
Epoch 1/5
100000/100000 [==============================] - 4s - loss: 0.0141 - acc: 0.9768 - val_loss: 6.5100e-06 - val_acc: 1.0000
Epoch 2/5
100000/100000 [==============================] - 4s - loss: 5.9377e-06 - acc: 1.0000 - val_loss: 5.5847e-06 - val_acc: 1.0000
Epoch 3/5
100000/100000 [==============================] - 4s - loss: 5.2044e-06 - acc: 1.0000 - val_loss: 4.9556e-06 - val_acc: 1.0000
Epoch 4/5
100000/100000 [==============================] - 5s - loss: 4.6606e-06 - acc: 1.0000 - val_loss: 4.4707e-06 - val_acc: 1.0000
Epoch 5/5
100000/100000 [==============================] - 4s - loss: 4.2337e-06 - acc: 1.0000 - val_loss: 4.0842e-06 - val_acc: 1.0000


test

In [33]:
testx = np.array([[0, 1]])
model.predict(testx)

array([[ 0.99886477]])

Synthetic Weather Symbol 
=============

In [13]:
c_dict = {"sun":[1,0,0], "sun with clouds":[0,1,0], "clouds":[0,0,1]}
p_dict = {"":[1,0,0], "rain":[0,1,0], "heavy rain":[0,0,1]}

In [15]:
def lookup(vec, ldict):
    result = None
    for k,v in ldict.items():
        if v == vec:
            result = k
    return result

In [3]:
def convert(p, q):
    ret = [0,0,0,0,0,0,0,0,0]
    place = p.index(1) + 3*q.index(1)
    ret[place] = 1
    return ret

In [53]:
def toWords(vec):
    i = vec.index(max(vec))
    print vec
    print max(vec)
    print i
    ci = i/3
    pi = i%3
    c = [0,0,0]
    p = [0,0,0]
    c[ci] = 1
    p[pi] = 1
    return lookup(c, c_dict) + ", " + lookup(p, p_dict)

In [4]:
ls = [[1,0,0], [0,1,0], [0,0,1]]
for a in ls:
    for b in ls:
        print convert(a, b)

[1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1]


In [5]:
def getSymbol(cf, pc):
    a = ""
    b = ""
    p = None
    q = None
    
    if 0.0 < cf <= 0.3:
        a = "sun"
        p = [1,0,0]
    elif 0.3 < cf <= 0.6:
        a = "sun with clouds"
        p = [0,1,0]
    elif 0.6 < cf:
        a = "clouds"
        p = [0,0,1]
        
    if 0.0 < pc <= 0.1:
        b = ""
        q = [1,0,0]
    elif 0.1 < pc <= 0.6:
        b = "rainy"
        q = [0,1,0]
    elif 0.6 < pc:
        b = "very rainy"
        q = [0,0,1]
        
#     print cf, pc, a, b
    return convert(p, q)

In [6]:
import numpy as np

ndata = 1000000

obs = []
symbols = []

for i in range(ndata):
    cf = np.random.rand()
    pc = np.random.rand() * cf
    
    obs.append([cf, pc])
    symbols.append(getSymbol(cf, pc))

In [7]:
np.array(symbols).shape

(1000000, 9)

In [8]:
model = Sequential()

model.add(Dense(6, input_dim=2))
model.add(Activation('sigmoid'))
model.add(Dense(9))
model.add(Activation('softmax'))


sgd = SGD(lr=0.1, decay=1e-6, momentum=0.99, nesterov=True)
model.compile(loss='mse', optimizer=sgd)

In [ ]:
model.fit(np.array(obs), np.array(symbols),
          nb_epoch=5,
          batch_size=100,
          show_accuracy=True,
          validation_split=0.1)

Train on 900000 samples, validate on 100000 samples
Epoch 1/5
900000/900000 [==============================] - 14s - loss: 0.0264 - acc: 0.8663 - val_loss: 0.0158 - val_acc: 0.9264
Epoch 2/5
900000/900000 [==============================] - 13s - loss: 0.0146 - acc: 0.9295 - val_loss: 0.0138 - val_acc: 0.9323
Epoch 3/5
900000/900000 [==============================] - 14s - loss: 0.0132 - acc: 0.9337 - val_loss: 0.0128 - val_acc: 0.9369
Epoch 4/5
417300/900000 [============>.................] - ETA: 6s - loss: 0.0128 - acc: 0.9350

In [72]:
testx = np.array([[0.5,0.9]])
r = model.predict(testx)
print r
print toWords(r.tolist()[0])

[[  2.01460646e-20   1.00291529e-11   4.76626560e-09   7.04265083e-04
    1.05666816e-02   7.46127000e-08   5.97862093e-10   5.95798244e-10
    9.88729000e-01]]
[2.0146064598085684e-20, 1.00291528554175e-11, 4.766265604416731e-09, 0.0007042650831863284, 0.010566681623458862, 7.461270001840603e-08, 5.978620931657019e-10, 5.95798244074075e-10, 0.9887290000915527]
0.988729000092
8
clouds, heavy rain
